# Introduction

This notebook presents a benchmark comparison between `power-grid-model` and 
[`pandapower`](https://github.com/e2nIEE/pandapower).
It runs several calculations, measures the calculation time, and compares the results.

## Test Network

The test network is fictionally generated using pre-defined random criteria. It is a radial network as follows:

```

source --- source_node ---| ---line--- node ---line--- node ...   (n_node_per_feeder)
                          |              |              |
                          |            load            load ...
                          |
                          | ---line--- ...
                          | .
                          | .
                          | .
                          | (n_feeder)

```

There is a node which is connected to a source (external network). From the source node there are `n_feeder` feeders. For each feeder there are `n_node_per_feeder` nodes, lines, and asymmetric loads. There are in total `n_feeder * n_node_per_feeder + 1` nodes in the network.


## Calculation

The notebook runs a power flow calculation with the same input data in `power-grid-model` and `pandapower`. It runs the following calculations:

* Single calculation with solver initialization.
* Single calculation without solver initialization (using pre-cached internal matrices).
* Time-series calculation
* N-1 calculation

The calculation is run symmetrically and asymmetrically. Both `power-grid-model` and `pandapower` supports asymmetric loads in symmetric calculations: the three-phase load is aggregated into one symmetric load.

We use the Newton-Raphson method for both libraries. In addition, we also use the iterative current and linear method in `power-grid-model` to see how much performance you can gain in exchange for accuracy.

## Results Comparison

The following results are compared between `power-grid-model` and `pandapower`:

* Per unit voltage of nodes (buses). For asymmetric calculation, it compares the value per phase.
* Loading of the lines.

It only compares the results of the Newton-Raphson, since the linear method of `power-grid-model` will produce a less accurate result.

## Performance Benchmark

The CPU time is measured for the **calculation part** of the program. The data preparation and model initialization is not measured. Furthermore, the single calculation is benchmarked with and without solver initialization. The former needs to execute connectivity check and initialize internal matrices (for example node admittance matrix). The latter uses the pre-cached connectivity and internal matrices.

# Preparation

## Import Libraries

We import neede libraries here. The fictional network generation and time-series profile generation is in a different Python file
[generate_fictional_dataset.py](./generate_fictional_dataset.py).

In [1]:
import tempfile
import time
from pathlib import Path

import numpy as np
import pandapower as pp
import pandas as pd
import power_grid_model as pgm

from generate_fictional_dataset import generate_fictional_grid, generate_time_series

# output dir for pandapower time-series
output_dir = Path(tempfile.gettempdir()) / "pandapower_time_series"
output_dir.mkdir(exist_ok=True)

## Prepare Tables

The performance comparison table and result deviation table is initialized below.

In [2]:
# summary
summary_df = pd.DataFrame(
    np.zeros(shape=(8, 5), dtype=np.float64),
    columns=['PGM Linear Impedance', 'PGM Linear Current' ,'PGM Newton-Raphson', 'PGM Iterative Current', 'PandaPower Newton-Raphson'],
    index=[
        'Symmetric calculation with solver initialization',
        'Symmetric calculation without solver initialization',
        'Asymmetric calculation with solver initialization',
        'Asymmetric calculation without solver initialization',
        'Time series symmetric calculation',
        'Time series asymmetric calculation',
        'N-1 symmetric calculation',
        'N-1 asymmetric calculation',
    ]
)

comparison_df = pd.DataFrame(
    np.zeros(shape=(6, 2), dtype=np.float64),
    columns=['Deviation Voltage (p.u.)', 'Deviation Loading (p.u.)'],
    index=[
        'Symmetric calculation',
        'Asymmetric calculation',
        'Time series symmetric calculation',
        'Time series asymmetric calculation',
        'N-1 symmetric calculation',
        'N-1 asymmetric calculation',
    ]
)

## Simulation Parameters

The simulation parameters, for example, the total number of feeders `n_feeder`, are defined below.

In [3]:
# fictional grid parameters

n_node_per_feeder = 10
n_feeder = 100

cable_length_km_min = 0.8
cable_length_km_max = 1.2
load_p_w_max = 0.4e6 * 0.8
load_p_w_min = 0.4e6 * 1.2
pf = 0.95

load_scaling_min = 0.5
load_scaling_max = 1.5
n_step = 1000
n_node = n_node_per_feeder * n_feeder + 1
n_line = n_node_per_feeder * n_feeder
n_load = n_node_per_feeder * n_feeder

use_lightsim2grid = True

## Pre-cache Library

To make a fair comparison, we run one small network so that both `power-grid-model` and `pandapower` can cache their dependent libraries into the memory.

* For `pandapower` the `numba` functions are JIT compiled and cached in the memory.

In [4]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=3,
    n_feeder=2,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp.runpp(fictional_dataset['pp_net'], algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, 
         lightsim2grid=use_lightsim2grid)
pgm_model = pgm.PowerGridModel(fictional_dataset['pgm_dataset'])
pgm_result = pgm_model.calculate_power_flow()

# Generate Dataset

First generate the fictional datasets.

In [5]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp_net = fictional_dataset['pp_net']
pgm_dataset = fictional_dataset['pgm_dataset']

# Benchmark Function

In [6]:
def benchmark_pgm_power_flow(symmetric: bool, calculation_type: str, update_data=None, with_intialization: bool = False):
    method_dict = {
        'linear': 'PGM Linear Impedance',
        'linear_current': 'PGM Linear Current',
        'iterative_current': 'PGM Iterative Current',
        'newton_raphson': 'PGM Newton-Raphson'
    }
    for method in ['linear', 'linear_current', 'iterative_current', 'newton_raphson']:
        model_instance = pgm.PowerGridModel(pgm_dataset)
        # cachec internal state if we do not benchmark solver intialization
        if not with_intialization:
            model_instance.calculate_power_flow(symmetric=symmetric, calculation_method=method)
        start = time.time()
        pgm_result = model_instance.calculate_power_flow(symmetric=symmetric, calculation_method=method, update_data=update_data)
        end = time.time()
        summary_df.loc[calculation_type, method_dict[method]] = end - start
    return pgm_result

# Single Calculation

We begin with a single power flow calculation.

## Symmetric

### power-grid-model

In [7]:
benchmark_pgm_power_flow(symmetric=True, calculation_type='Symmetric calculation with solver initialization', with_intialization=True)
pgm_result = benchmark_pgm_power_flow(symmetric=True, calculation_type='Symmetric calculation without solver initialization', with_intialization=False)


### Newton-Raphson Method of pandapower

In [8]:
# first calculation with solver initialization
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PandaPower Newton-Raphson'] = end - start

# second calculation with existing solver
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PandaPower Newton-Raphson'] = end - start

### Calculate Deviation for Newton-Raphson Method

In [9]:
comparison_df.loc['Symmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_net.res_bus['vm_pu'] - pgm_result['node']['u_pu']).max()
comparison_df.loc['Symmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_net.res_line['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

## Asymmetric

### power-grid-model

In [10]:
benchmark_pgm_power_flow(symmetric=False, calculation_type='Asymmetric calculation with solver initialization', with_intialization=True)
pgm_result = benchmark_pgm_power_flow(symmetric=False, calculation_type='Asymmetric calculation without solver initialization', with_intialization=False)

### Newton-Raphson Method of pandapower

In [11]:
# first calculation with solver initialization
start = time.time()
pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PandaPower Newton-Raphson'] = end - start

# second calculation with existing solver
start = time.time()
pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PandaPower Newton-Raphson'] = end - start


### Calculate Deviation for Newton-Raphson Method

In [12]:
comparison_df.loc['Asymmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']].to_numpy() - pgm_result['node']['u_pu']).max()
comparison_df.loc['Asymmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_net.res_line_3ph['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

# Time Series Calculation

We execute a time-series power flow with `n_step` timestamps. 

## Preparation

The load profile is randomly generated by the function `generate_time_series`. It produces the relevant input format for both libraries.

In [13]:
time_series_dataset = generate_time_series(
    fictional_dataset,
    n_step=n_step,
    load_scaling_min=load_scaling_min,
    load_scaling_max=load_scaling_max
)

pgm_update_dataset = time_series_dataset['pgm_update_dataset']
pp_dataset = time_series_dataset['pp_dataset']
time_steps = np.arange(n_step)

for x, y in zip(['p', 'q'], ['mw', 'mvar']):
    for p in ['a', 'b', 'c']:
        name = f'{x}_{p}_{y}'
        pp.control.ConstControl(
            pp_net,
            element='asymmetric_load',
            element_index=pp_net.asymmetric_load.index,
            variable=name,
            data_source=pp_dataset[name],
            profile_name=pp_net.asymmetric_load.index
        )


## Symmetric

### power-grid-model

In [14]:
pgm_result = benchmark_pgm_power_flow(symmetric=True, calculation_type='Time series symmetric calculation', update_data=pgm_update_dataset)



### Newton-Raphson Method of pandapower

In [15]:
pp.timeseries.OutputWriter(
    pp_net, output_path=output_dir, output_file_type=".csv", csv_separator=',',
    log_variables=[
        ('res_bus', 'vm_pu'),
        ('res_line', 'loading_percent'),
    ]
)

start = time.time()
pp.timeseries.run_timeseries(
    pp_net, run=pp.runpp, time_steps=time_steps,
    calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid
)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PandaPower Newton-Raphson'] = end - start

100%|██████████| 1000/1000 [00:21<00:00, 45.66it/s]


### Calculate Deviation for Newton-Raphson Method

In [16]:
pp_u_pu = pd.read_csv(output_dir / 'res_bus' / 'vm_pu.csv', index_col=0).to_numpy()
comparison_df.loc['Time series symmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()
pp_loading = pd.read_csv(output_dir / 'res_line' / 'loading_percent.csv', index_col=0).to_numpy() * 1e-2
comparison_df.loc['Time series symmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()


## Asymmetric

### power-grid-model

In [17]:
pgm_result = benchmark_pgm_power_flow(symmetric=False, calculation_type='Time series asymmetric calculation', update_data=pgm_update_dataset)


### Newton-Raphson Method of pandapower

In [18]:
del pp_net.output_writer
ow = pp.timeseries.OutputWriter(pp_net, output_path=output_dir, output_file_type=".csv", csv_separator=',')
ow.log_variable('res_bus_3ph', 'vm_a_pu', index=pp_net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_b_pu', index=pp_net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_c_pu', index=pp_net.bus.index)
ow.log_variable('res_line_3ph', 'loading_percent', index=pp_net.line.index)

# run
start = time.time()
pp.timeseries.run_timeseries(
    pp_net,
    run=pp.runpp_3ph,
    time_steps=time_steps,
    calculate_voltage_angles=True,
    distributed_slack=True, 
    lightsim2grid=use_lightsim2grid
)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PandaPower Newton-Raphson'] = end - start

100%|██████████| 1000/1000 [04:29<00:00,  3.71it/s]


### Calculate Deviation for Newton-Raphson Method

In [19]:
pp_u_pu = []
for p in ['a', 'b', 'c']:
    pp_u_pu.append(pd.read_csv(output_dir / 'res_bus_3ph' / f'vm_{p}_pu.csv', index_col=0).to_numpy())
pp_u_pu = np.stack(pp_u_pu, axis=-1)
comparison_df.loc['Time series asymmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()
pp_loading = pd.read_csv(output_dir / 'res_line_3ph' / 'loading_percent.csv', index_col=0).to_numpy() * 1e-2
comparison_df.loc['Time series asymmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()

# N-1 Scenario Calculation

We execute a N-1 scenario calculation. There are `n_line` scenarios. In each scenario one line will be disabled. Since the original network is radial, part of the network will be unenergized due to the switch-off of a line.

## Preparation

The N-1 scenario input is generated for `power-grid-model`. Since `pandapower` does not have a built-in N-1 scenario calculation, we execute the power flow per scenario in a loop.

In [20]:
# re-generate dataset
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp_net = fictional_dataset['pp_net']
pgm_dataset = fictional_dataset['pgm_dataset']

# update dataset for power grid model
# disable one line per batch
pgm_line_profile = pgm.initialize_array('update', 'line', (n_line, n_line))
pgm_line_profile['id'] = pgm_dataset['line']['id']
pgm_line_profile['from_status'] = 1
pgm_line_profile['to_status'] = 1
np.fill_diagonal(pgm_line_profile['from_status'], 0)
np.fill_diagonal(pgm_line_profile['to_status'], 0)
pgm_update_dataset = {'line': pgm_line_profile}




## Symmetric

### power-grid-model

In [21]:
pgm_result = benchmark_pgm_power_flow(symmetric=True, calculation_type='N-1 symmetric calculation', update_data=pgm_update_dataset)


### Newton-Raphson Method of pandapower

In [22]:
# prepare pandapower result dataset
pp_u_pu = np.empty(shape=(n_line, n_node), dtype=np.float64)
pp_loading = np.empty(shape=(n_line, n_line), dtype=np.float64)

start = time.time()
# loop to calcualte pandapower N-1
for i in pp_net.line.index:
    # set one line out of service
    pp_net.line.loc[i, 'in_service'] = False
    pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
    # restore that line
    pp_net.line.loc[i, 'in_service'] = True
    # get result
    pp_u_pu[i, :] = pp_net.res_bus['vm_pu']
    pp_loading[i, :] = pp_net.res_line['loading_percent'] * 1e-2
end = time.time()
summary_df.loc['N-1 symmetric calculation', 'PandaPower Newton-Raphson'] = end - start

# set nan to 0.0 to make a meaningful comparison
pp_u_pu[np.isnan(pp_u_pu)] = 0.0
pp_loading[np.isnan(pp_loading)] = 0.0

### Calculate Deviation for Newton-Raphson Method

In [23]:
comparison_df.loc['N-1 symmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()
comparison_df.loc['N-1 symmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()

## Asymmetric

### power-grid-model

In [24]:
pgm_result = benchmark_pgm_power_flow(symmetric=False, calculation_type='N-1 asymmetric calculation', update_data=pgm_update_dataset)


### Newton-Raphson Method of pandapower

In [25]:
%%capture

# prepare pandapower result dataset
pp_u_pu = np.empty(shape=(n_line, n_node, 3), dtype=np.float64)
pp_loading = np.empty(shape=(n_line, n_line), dtype=np.float64)

start = time.time()
# loop to calcualte pandapower N-1
for i in pp_net.line.index:
    # set one line out of service
    pp_net.line.loc[i, 'in_service'] = False
    pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
    # restore that line
    pp_net.line.loc[i, 'in_service'] = True
    # get result
    pp_u_pu[i, ...] = pp_net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']]
    pp_loading[i, :] = pp_net.res_line_3ph['loading_percent'] * 1e-2
end = time.time()
summary_df.loc['N-1 asymmetric calculation', 'PandaPower Newton-Raphson'] = end - start

# set nan to 0.0 to make a meaningful comparison
pp_u_pu[np.isnan(pp_u_pu)] = 0.0
pp_loading[np.isnan(pp_loading)] = 0.0

### Calculate Deviation for Newton-Raphson Method

In [26]:
comparison_df.loc['N-1 asymmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()
comparison_df.loc['N-1 asymmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()

# Summary

## Deviation of Results

Below is the table of deviation between the results from `power-grid-model` and `pandapower`. It matches to the order of `1e-7`.

In [27]:
display(comparison_df)

,Deviation Voltage (p.u.),Deviation Loading (p.u.)
Symmetric calculation,1.628697e-12,5.208611e-13
Asymmetric calculation,3.299400e-08,1.606717e-07
Time series symmetric calculation,1.781686e-12,1.266542e-12
Time series asymmetric calculation,1.404013e-07,6.695306e-07
N-1 symmetric calculation,1.641576e-12,5.788148e-13
N-1 asymmetric calculation,3.299403e-08,1.606718e-07


## Performance Comparison

Below is the table of the measured time of all calculations.

In [28]:
display(summary_df)

,PGM Linear Impedance,PGM Linear Current,PGM Newton-Raphson,PGM Iterative Current,PandaPower Newton-Raphson
Symmetric calculation with solver initialization,0.000642,0.000581,0.001200,0.000670,0.025094
Symmetric calculation without solver initialization,0.000256,0.000277,0.000746,0.000453,0.017335
Asymmetric calculation with solver initialization,0.002624,0.001994,0.005784,0.002526,1.078073
Asymmetric calculation without solver initialization,0.002445,0.000851,0.003941,0.001129,0.208731
Time series symmetric calculation,0.281519,0.268539,0.729586,0.443939,21.912689
Time series asymmetric calculation,1.014977,0.480715,3.430360,0.954616,269.506399
N-1 symmetric calculation,0.490580,0.531929,0.989949,0.700397,16.167235
N-1 asymmetric calculation,1.313481,1.373128,3.751887,1.759124,209.205484


In [29]:
relative_df = summary_df.div(summary_df.max(axis=1), axis=0)
speedup_df = 1 / relative_df
display(speedup_df.style.format("{:0.2f}x"))

,PGM Linear Impedance,PGM Linear Current,PGM Newton-Raphson,PGM Iterative Current,PandaPower Newton-Raphson
Symmetric calculation with solver initialization,39.07x,43.21x,20.91x,37.44x,1.00x
Symmetric calculation without solver initialization,67.64x,62.52x,23.24x,38.23x,1.00x
Asymmetric calculation with solver initialization,410.88x,540.75x,186.39x,426.86x,1.00x
Asymmetric calculation without solver initialization,85.35x,245.23x,52.97x,184.86x,1.00x
Time series symmetric calculation,77.84x,81.60x,30.03x,49.36x,1.00x
Time series asymmetric calculation,265.53x,560.64x,78.57x,282.32x,1.00x
N-1 symmetric calculation,32.96x,30.39x,16.33x,23.08x,1.00x
N-1 asymmetric calculation,159.28x,152.36x,55.76x,118.93x,1.00x
